In [1]:
from numpy import linspace, sin, cos, pi, polyfit
from devito import SpaceDimension, Constant, Grid, TimeFunction, Eq, Operator, configuration
from sympy import solve
from scipy.integrate import quad
import matplotlib.pyplot as plt
#configuration['log-level'] = 'WARNING'


# Set global constants
L = 10. # Define length of domain as a global variable
k = 100 # Number of terms in the Fourier sine series

# Set up FD parameters
so_dev = 4 # 4th order accurate in space for conventional
so_opt = 4 # 4th order accurate in space for optimized
to = 2 # 2nd order accurate in time
t_end = L # Standing wave will cycle twice in time L
# Optimized stencil coefficients
a_0 = -2.81299833
a_1 = 1.56808208
a_2 = -0.17723283
a_3 = 0.01564992
extent = (L,) # Grid is L long with l grid points


# Functions for initalizing standing square wave/Calculating u(x, t)
def square_init(x): # Square function to base Fourier series off of
    if x >= 0 and x < L/4.:
        return 1.
    elif x >= L/4. and x < L/2.:
        return -1.
    else:
        return 0.

def D_n_inner(x, n): # Inner part of D_n to allow for scipy.integrate.quad to be used
    return square_init(x)*sin(n*pi*x/L)
    
def D_n_calc(n): # Define function to calculate d_n for a given n
    if n % 2 == 0:
        return (4./L)*quad(D_n_inner, 0, L/2., args=(n))[0]
    else:
        return 0.

def u(x, t): # Analytic solution calculator
    u_temp = 0.
    
    for n in range(1, 2*k+1):
        u_temp += D_n_calc(n)*sin(n*pi*x/L)*cos(n*pi*t/L)
        
    return u_temp


dx_vals = [] # List to store dx
max_err_dev = [] # Lists to store max(abs(u_numerical - u_analytic))
max_err_opt = []


# Solve for time t = L at various dx with optimized and conventional methods
for points in range(701, 1102, 100):
    l = points
    x_vals = linspace(0, L, l) # x axis for calling u(x, t) at given t
    shape = (l,)
    dt = 0.01*(L/(l-1)) # Timestep is defined relative to critical dt
    ns = int(t_end/dt) # Number of timesteps = total time/timestep size
    
    x = SpaceDimension(name='x', spacing=Constant(name='h_x', value=extent[0]/(shape[0]-1)))

    grid = Grid(extent=extent, shape=shape, dimensions=(x,))
    time = grid.time_dim
    h_x = grid.spacing[0]
    
    # Set up function and stencil for conventional solution
    u_dev = TimeFunction(name="u_dev", grid=grid, space_order=so_dev, time_order=to, save=ns+1)
    stencil_dev = Eq(u_dev.forward, solve(u_dev.dx2 - u_dev.dt2, u_dev.forward)[0])
    bc_dev = [Eq(u_dev[time+1,0], 0.0)] # Specify Dirichlet boundary conditions
    bc_dev += [Eq(u_dev[time+1,-1], 0.0)]
    
    # Set up function and stencil
    u_opt = TimeFunction(name="u_dev", grid=grid, space_order=so_opt, time_order=to, save=ns+1)
    stencil_opt = Eq(u_opt.forward, solve((a_3*u_opt[time, x - 3]
                                          + a_2*u_opt[time, x - 2]
                                          + a_1*u_opt[time, x - 1]
                                          + a_0*u_opt[time, x]
                                          + a_1*u_opt[time, x + 1]
                                          + a_2*u_opt[time, x + 2]
                                          + a_3*u_opt[time, x + 3])/h_x**2
                                          - u_opt.dt2, u_opt.forward)[0])

    bc_opt = [Eq(u_opt[time+1,0], 0.0)] # Specify boundary conditions
    bc_opt += [Eq(u_opt[time+1,-1], 0.0)]
    
    # Initialize wavefields
    u_dev.data[:] = u(x_vals, 0)
    u_opt.data[:] = u(x_vals, 0)
    # Create operators
    op_dev = Operator([stencil_dev] + bc_dev)
    op_opt = Operator([stencil_opt] + bc_opt)
    # Apply operators
    op_dev.apply(time_M=ns-1, dt=dt)
    op_opt.apply(time_M=ns-1, dt=dt)
    
    # Calculate numerical solution for t = L
    u_analytic = u(x_vals, L)
    
    # Append values to lists
    dx_vals.append(L/(l-1))
    max_err_dev.append(max(abs(u_dev.data[-2] - u_analytic)))
    max_err_opt.append(max(abs(u_opt.data[-2] - u_analytic)))
    
fig = plt.figure()
plt.plot(dx_vals, max_err_dev, 'k-', linewidth=0.75, label='Conventional 4th order')
plt.plot(dx_vals, max_err_opt, 'r-', linewidth=0.75, label='Optimized 4th order')
plt.title("Absolute maximum error vs grid spacing. Courant number = 0.01.")
plt.legend(loc='best')
plt.xlabel("Δx")
plt.ylabel("Max absolute error")
plt.savefig("Figures/Max_Absolute_Error/error_plot_1.9.png", dpi=200)
plt.show()

Operator `Kernel` run in 0.07 s
Operator `Kernel` run in 0.08 s
Operator `Kernel` run in 0.10 s
Operator `Kernel` run in 0.10 s
Operator `Kernel` run in 0.12 s
Operator `Kernel` run in 0.11 s
Operator `Kernel` run in 0.14 s
Operator `Kernel` run in 0.14 s
Operator `Kernel` run in 0.25 s
Operator `Kernel` run in 0.19 s


<Figure size 640x480 with 1 Axes>